# [Object Relational Tutorial](http://docs.sqlalchemy.org/en/latest/orm/tutorial.html)

The SQLAlchemy Object Relational Mapper presents a method of associating user-defined Python classes with database tables, and instances of those classes (objects) with rows in their corresponding tables.  
It includes a system that transparently synchronizes all changes in state between objects and their related rows, called a unit of work, as well as a system for expressing database queries in terms of the user defined classes and their defined relationships between each other.  
The ORM is in contrast to the SQLAlchemy Expression Language, upon which the ORM is constructed.  
Whereas the SQL Expression Language, introduced in SQL Expression Language Tutorial, presents a system of representing the primitive constructs of the relational database directly without opinion, the ORM presents a high level and abstracted pattern of usage, which itself is an example of applied usage of the Expression Language.  
While there is overlap among the usage patterns of the ORM and the Expression Language, the similarities are more superficial than they may at first appear.  
One approaches the structure and content of data from the perspective of a user-defined domain model which is transparently persisted and refreshed from its underlying storage model.  
The other approaches it from the perspective of literal schema and SQL expression representations which are explicitly composed into messages consumed individually by the database.  
A successful application may be constructed using the Object Relational Mapper exclusively.  
In advanced situations, an application constructed with the ORM may make occasional usage of the Expression Language directly in certain areas where specific database interactions are required.

In [17]:
import sqlalchemy
sqlalchemy.__version__

'1.2.10'

For this tutorial we will use an in-memory-only SQLite database.  
To connect we can use `create_engine()`:

In [18]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True); engine.__doc__

'\n    Connects a :class:`~sqlalchemy.pool.Pool` and\n    :class:`~sqlalchemy.engine.interfaces.Dialect` together to provide a\n    source of database connectivity and behavior.\n\n    An :class:`.Engine` object is instantiated publicly using the\n    :func:`~sqlalchemy.create_engine` function.\n\n    See also:\n\n    :doc:`/core/engines`\n\n    :ref:`connections_toplevel`\n\n    '

In [19]:
engine

Engine(sqlite:///:memory:)

The echo flag is a shortcut to setting up SQLAlchemy logging, which is accomplished via Python’s standard logging module.  
With it enabled, we’ll see all the generated SQL produced.  
If you are working through this tutorial and want less output generated, set it to False.  
This tutorial will format the SQL behind a popup window so it doesn’t get in our way; just click the “SQL” links to see what’s being generated.  
The return value of `create_engine()` is an instance of Engine, and it represents the core interface to the database, adapted through a dialect that handles the details of the database and DBAPI in use.  
In this case the SQLite dialect will interpret instructions to the Python built-in sqlite3 module.  
The first time a method like `Engine.execute()` or `Engine.connect()` is called, the Engine establishes a real DBAPI connection to the database, which is then used to emit the SQL.  
When using the ORM, we typically don’t use the Engine directly once created; instead, it’s used behind the scenes by the ORM as we’ll see shortly.  
**Note about Lazy Connecting:**  
The Engine, when first returned by `create_engine()`, has not actually tried to connect to the database yet; that happens only the first time it is asked to perform a task against the database.

## [Declare a Mapping](http://docs.sqlalchemy.org/en/latest/orm/tutorial.html#declare-a-mapping)

When using the ORM, the configurational process starts by describing the database tables we’ll be dealing with, and then by defining our own classes which will be mapped to those tables.  
In modern SQLAlchemy, these two tasks are usually performed together, using a system known as Declarative, which allows us to create classes that include directives to describe the actual database table that they will be mapped to.  
Classes mapped using the Declarative system are defined in terms of a base class which maintains a catalog of classes and tables relative to that base - this is known as the declarative base class.  
Our application will usually have just one instance of this base in a commonly imported module.  
We create the base class using the `declarative_base()` function, as follows:

In [20]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base(); Base.__doc__

'The most base type'

In [21]:
Base

sqlalchemy.ext.declarative.api.Base

Now that we have a base of the most base type, we can define any number of mapped classes in terms of it.  
We will start with just a single table called **users**, which will store records for the end-users using our application.  
A new class called User will be the class to which we map this table.  
Within the class, we define details about the table to which we’ll be mapping, primarily the table name, and names and datatypes of columns.  
Although the User class defines a `__repr__()` method, it is optional; we only implement it in this tutorial so that our examples show nicely formatted User objects.

In [22]:
from sqlalchemy import Column, Integer, String


class User(Base):
    
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)
    
    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (self.name, self.fullname, self.password)

A class using Declarative at a minimum needs a `__tablename__` attribute, and at least one Column which is part of a primary key.  
SQLAlchemy never makes any assumptions by itself about the table to which a class refers, including that it has no built-in conventions for names, datatypes, or constraints.  
But this doesn’t mean boilerplate is required; instead, you’re encouraged to create your own automated conventions using helper functions and mixin classes, which is described in detail at Mixin and Custom Base Classes.  
When our class is constructed, Declarative replaces all the Column objects with special Python accessors known as descriptors; this is a process known as instrumentation.  
The "instrumented" mapped class will provide us with the means to refer to our table in a SQL context as well as to persist and load the values of columns from the database.  
Outside of what the mapping process does to our class, the class remains otherwise mostly a normal Python class, to which we can define any number of ordinary attributes and methods needed by our application.

## [Create a Schema](http://docs.sqlalchemy.org/en/latest/orm/tutorial.html#create-a-schema)

With our User class constructed via the Declarative system, we have defined information about our table, known as table metadata.  
The object used by SQLAlchemy to represent this information for a specific table is called the Table object, and here Declarative has made one for us.  
We can see this object by inspecting the `__table__` attribute:

When we declared our class, Declarative used a Python metaclass in order to perform additional activities once the class declaration was complete; within this phase, it then created a Table object according to our specifications, and associated it with the class by constructing a Mapper object.  
This object is a behind-the-scenes object we normally don’t need to deal with directly (though it can provide plenty of information about our mapping when we need it).  

The Table object is a member of a larger collection known as MetaData.  
When using Declarative, this object is available using the `.metadata` attribute of our declarative base class.  

The MetaData is a registry which includes the ability to emit a limited set of schema generation commands to the database.  
As our SQLite database does not actually have a users table present, we can use MetaData to issue `CREATE TABLE` statements to the database for all tables that don’t yet exist.  
Below, we call the `MetaData.create_all()` method, passing in our Engine as a source of database connectivity.  
We will see that special commands are first emitted to check for the presence of the users table, and following that the actual `CREATE TABLE` statement:

In [24]:
Base.metadata.create_all(engine)

2018-08-13 02:35:04,782 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-08-13 02:35:04,783 INFO sqlalchemy.engine.base.Engine ()
2018-08-13 02:35:04,785 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-08-13 02:35:04,786 INFO sqlalchemy.engine.base.Engine ()
2018-08-13 02:35:04,789 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2018-08-13 02:35:04,790 INFO sqlalchemy.engine.base.Engine ()
2018-08-13 02:35:04,794 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2018-08-13 02:35:04,795 INFO sqlalchemy.engine.base.Engine ()
2018-08-13 02:35:04,797 INFO sqlalchemy.engine.base.Engine COMMIT


**Classical Mappings**  
The Declarative system, though highly recommended, is not required in order to use SQLAlchemy’s ORM.  
Outside of Declarative, any plain Python class can be mapped to any Table using the `mapper()` function directly; this less common usage is described at [Classical Mappings](http://docs.sqlalchemy.org/en/latest/orm/mapping_styles.html#classical-mapping).


## [Create an Instance of the Mapped Class](http://docs.sqlalchemy.org/en/latest/orm/tutorial.html#create-an-instance-of-the-mapped-class)